In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
path = 'store-sales-time-series-forecasting/'

In [ ]:
train_df = pd.read_csv(path + 'train.csv',index_col='id')
test_df = pd.read_csv(path + 'test.csv',index_col='id')

# Explanatory Data Analysis

In [ ]:
#change date datatype as datetime
train_df.date = pd.to_datetime(train_df.date)
test_df.date= pd.to_datetime(test_df.date)

train_df['year'] = train_df.date.dt.year
test_df['year'] = test_df.date.dt.year

train_df['month'] = train_df.date.dt.month
test_df['month'] = test_df.date.dt.month

train_df['dayofmonth'] = train_df.date.dt.day
test_df['dayofmonth'] = test_df.date.dt.day

train_df['dayofweek'] = train_df.date.dt.dayofweek
test_df['dayofweek'] = test_df.date.dt.dayofweek

train_df['dayname'] = train_df.date.dt.strftime('%A')
test_df['dayname'] = test_df.date.dt.strftime('%A')

<font size=4>In this competition, need to predict the sales of stores the last two weeks of August 2017. <font>

# Oil CSV

In [ ]:
import datetime

oil = pd.read_csv(path + 'oil.csv',index_col='date')
#some days are skipped. Filling up the gap.

start_date = train_df.date.min() 
# from beggining of the train date and the end of test date
number_of_days = 1704 #1703
date_list = [(start_date + datetime.timedelta(days = day)).isoformat() for day in range(number_of_days)]

date = (pd.Series(date_list)).to_frame()
date.columns = ['date']
date.date = pd.to_datetime(date.date)
date['date_str'] = date.date.astype(str)
oil['date_str'] = oil.index.astype(str)

oil = pd.merge(date,oil,how='left',on='date_str')



In [ ]:
oil = oil.set_index('date').dcoilwtico.interpolate(method='linear').to_frame()
#oil.set_index('date').dcoilwtico.plot();



In [ ]:
oil['date_str'] = oil.index.astype(str)


In [ ]:
# add oil price to the train date
train_df['date_str'] = train_df.date.astype(str)

train_df = pd.merge(train_df,oil,how='left',on='date_str')
train_df.drop(columns='date_str',axis=1,inplace=True)


In [ ]:
test_df['date_str'] = test_df.date.astype(str)
test_df = pd.merge(test_df,oil,how='left', on='date_str')

test_df.drop(columns='date_str',axis=1,inplace=True)

<font size =4> Some product family (Home Care,Magazines,Home and Kitchen II,Produce,Home and Kitchen I)<br>
    have slightly stronger correlationship with the oil price.
    </font>

# Stores CSV

In [ ]:
stores = pd.read_csv(path + 'stores.csv',index_col='store_nbr')
train_df = pd.merge(train_df,stores,how='left',on='store_nbr')
test_df = pd.merge(test_df,stores,how='left',on='store_nbr')

# Holidays_Events.CSV

In [ ]:
holiday_df = pd.read_csv(path + 'holidays_events.csv')
holiday_df.query('transferred==True')

# transferred day is not celebrated
holiday_df = holiday_df.query('transferred ==False')
holiday_df.description = holiday_df.description.str.replace('Traslado ','')

#national
national = holiday_df.query('locale=="National"')

day_off = national.query('type!="Work Day" or type!="Event"').set_index('date')['description'].to_dict()

train_df['date_str'] = train_df.date.astype(str)
test_df['date_str'] = test_df.date.astype(str)

train_df['national_holiday'] = [1 if a in day_off else 0 for a in train_df.date_str]
test_df['national_holiday'] = [1 if a in day_off else 0 for a in test_df.date_str]

event = national.query('type=="Event"').set_index('date')['description'].to_dict()

train_df['national_event'] =[1 if a in event else 0 for a in train_df.date_str]
test_df['national_event'] =[1 if a in event else 0 for a in test_df.date_str]

work_day = national.query('type=="Work Day"').set_index('date')['description'].to_dict()
train_df['national_workday'] = [1 if a in work_day else 0 for a in train_df.date_str]
test_df['national_workday'] = [1 if a in work_day else 0 for a in test_df.date_str]

#weekend
train_df['weekend'] = [1 if a>=5 else 0 for a in train_df.dayofweek]
test_df['weekend'] = [1 if a>=5 else 0 for a in test_df.dayofweek]

In [ ]:
train_df.columns

In [ ]:
#locale
#locale is corresponding to city of train_df
local = holiday_df.query('locale=="Local"')
local_dic = local.set_index('date').locale_name.to_dict()
train_df['local_holiday']=[1 if b in local_dic and local_dic[b]== a else 0 for a,b in zip(train_df.city,train_df.date_str)]
test_df['local_holiday']=[1 if b in local_dic and local_dic[b]== a else 0 for a,b in zip(test_df.city,test_df.date_str)]

In [ ]:
#Regional
#Regional is corresonding to state of train_df
regional = holiday_df.query('locale=="Regional"')
regional_dic = regional.set_index('date').locale_name.to_dict()
train_df['regional_holiday']= [1 if b in regional_dic and regional_dic[b]== a else 0 for a,b in zip(train_df.state,train_df.date_str)]
test_df['regional_holiday']= [1 if b in regional_dic and regional_dic[b]== a else 0 for a,b in zip(test_df.state,test_df.date_str)]



# Transaction.CSV 

In [ ]:
transaction_df = pd.read_csv(path + 'transactions.csv')

<fond size=4>
    Every store have peak days in Christmas season. Some stores show upper trends.

# Preparing Data

In [ ]:
len(train_df.query('date_str=="2013-01-01"'))

In [ ]:
#https://www.kaggle.com/c/store-sales-time-series-forecasting/discussion/291165
# idea and codes comes from this discussion 

train_df.sales = np.log1p(train_df.sales)


In [ ]:
train_df['Istest'] = False
test_df['Istest'] = True

full = pd.concat((train_df,test_df))

#remove leap year day
#full = full.query('date_str !="2016-02-29"')


full['Lag_16'] = full['sales'].shift(1782*16)
full['Lag_17'] = full['sales'].shift(1782*17)
full['Lag_18'] = full['sales'].shift(1782*18)
full['Lag_19'] = full['sales'].shift(1782*19)
full['Lag_20'] = full['sales'].shift(1782*20)
full["Lag_21"] = full['sales'].shift(1782*21)
full['Lag_22'] = full['sales'].shift(1782*22)
full['Lag_28'] = full['sales'].shift(1782*28)
full['Lag_31'] = full['sales'].shift(1782*31)

full['Lag_365'] = full['sales'].shift(1782*365)


train_df = full.query('Istest==False')
test_df = full.query('Istest ==True')


In [ ]:
train_df = train_df.dropna(subset=['Lag_365'],axis=0)

In [ ]:
train_df

In [ ]:
FEATURES = ['store_nbr','family','onpromotion', 'year', 'month',
       'dayofmonth', 'dayofweek','dcoilwtico', 'city', 'state',
       'type', 'cluster', 'national_holiday', 'national_event',
       'national_workday', 'weekend', 'local_holiday', 'regional_holiday','Lag_16','Lag_17','Lag_18','Lag_19','Lag_20','Lag_21','Lag_22','Lag_28','Lag_31','Lag_365']
TARGET =['sales']



In [ ]:
from sklearn import preprocessing
categories = ['family','city','state','type']
for i in categories:
    encoder = preprocessing.LabelEncoder()
    train_df[i] = encoder.fit_transform(train_df[i])
    test_df[i] =  encoder.transform(test_df[i])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(train_df,train_df[TARGET],test_size=0.05,shuffle=False)

In [ ]:
y_train

In [ ]:
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import mean_squared_log_error

linear = LinearRegression()
model = linear.fit(X_train[FEATURES],y_train)


#plt.scatter(X_val.date,model.predict(X_val[FEATURES]))
#plt.xticks(rotation=45)

predictions= model.predict(X_val[FEATURES])
predictions = [a if a>0 else 0 for a in predictions]
print('MSLE: ' + str(mean_squared_log_error(y_val,predictions)))


In [ ]:
from trubrics.context import DataContext

In [ ]:
data_context = DataContext(
    testing_data=X_val[FEATURES].assign(sales=y_val),
    training_data=X_train[FEATURES].assign(sales=y_train),    
    target="sales"
)

In [ ]:
data_context.testing_data.head()

In [ ]:
from trubrics.validations import ModelValidator

In [ ]:
from sklearn.metrics import mean_squared_log_error, make_scorer

def custom_log_scorer(y_true, y_pred):
    y_pred = [a if a > 0 else 0 for a in y_pred]
    return mean_squared_log_error(y_true, y_pred)

In [ ]:
model_validator = ModelValidator(data=data_context, model=model, custom_scorers={"zero_neg_mean_squared_log_error": make_scorer(custom_log_scorer, greater_is_better=False)})

In [ ]:
validations = [
    model_validator.validate_inference_time(threshold=0.05, n_executions=1),
    model_validator.validate_performance_against_threshold(metric="zero_neg_mean_squared_log_error", threshold=-0.2),
    model_validator.validate_test_performance_against_dummy(metric="zero_neg_mean_squared_log_error", strategy="mean"),
    model_validator.validate_performance_between_train_and_test(metric="zero_neg_mean_squared_log_error", threshold=0.2),
]

In [ ]:
for validation in validations:
    print(validation.dict()["validation_type"], validation.dict()["outcome"], validation.dict()["result"])

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(tree_method='hist',n_estimators=500)
xgb.fit(
    X_train[FEATURES],
    y_train,
    eval_set=[(X_train[FEATURES],y_train),(X_val[FEATURES], y_val)],
    verbose=False,
    early_stopping_rounds=10
)

#plt.scatter(X_val.date,xgb.predict(X_val[FEATURES]))
#plt.xticks(rotation=45)

#predictions have negative. Changed negative values to 0.
predictions= xgb.predict(X_val[FEATURES])
predictions = [a if a>0 else 0 for a in predictions]
print('MSLE: ',mean_squared_log_error(y_val,predictions))

In [ ]:
model_validator = ModelValidator(data=data_context, model=xgb, custom_scorers={"zero_neg_mean_squared_log_error": make_scorer(custom_log_scorer, greater_is_better=False)})

In [ ]:
validations = [
    model_validator.validate_inference_time(threshold=0.05, n_executions=1),
    model_validator.validate_performance_against_threshold(metric="zero_neg_mean_squared_log_error", threshold=-0.2),
    model_validator.validate_test_performance_against_dummy(metric="zero_neg_mean_squared_log_error", strategy="mean"),
    model_validator.validate_performance_between_train_and_test(metric="zero_neg_mean_squared_log_error", threshold=0.2),
    model_validator.validate_feature_in_top_n_important_features(feature="Lag_16", top_n_features=2)
]

In [ ]:
for validation in validations:
    display(validation.dict()["validation_type"], validation.dict()["outcome"], validation.dict()["result"])
    print()

In [ ]:
fi = validations[-1].dict()["result"]["feature_importance"]
plt.figure(figsize=(10,10))
plt.barh(range(len(fi)), fi.values(), tick_label=list(fi.keys()))
plt.show()

In [ ]:
WRITE = False

if WRITE:
    import joblib
    joblib.dump(xgb, path + "xgb.joblib")
    data_context.testing_data.to_csv(path + "sample_x_test.csv", index=False)